# Functional Data Gathering - Areas

## Import modules

In [2]:
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from microns_phase3 import nda, utils

Connecting microns@db.datajoint.com:3306


## Building the dataframe of all functional cells

In [3]:
client = CAVEclient()

client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

#visualize available tables
client.materialize.get_tables()

['synapses_pni_2',
 'nucleus_neuron_svm',
 'nucleus_detection_v0',
 'functional_coreg',
 'allen_v1_column_types_slanted',
 'aibs_column_nonneuronal',
 'func_unit_em_match_release',
 'proofreading_status_public_release',
 'aibs_soma_nuc_metamodel_preds_v117']

In [4]:
df=client.materialize.query_table('functional_coreg') #dataframe of the functionally co-registered neurons
df.head() #visualize first couple neurons

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,pt_position
0,1,t,102531727972419182,864691136136642173,4,7,665,"[274864, 94064, 22046]"
1,2,t,94792540099826672,864691136208993724,4,7,740,"[218320, 104320, 19016]"
2,4,t,93946810036875732,864691135683934135,4,7,782,"[212128, 94544, 23423]"
3,5,t,84730565597057981,864691135334528233,4,7,855,"[145158, 110036, 19355]"
4,6,t,110905058841147285,864691136309730650,4,7,921,"[335744, 90128, 22805]"


This cell gives us the relevent identification values for each of the neurons in the co-registration subset so they can be used as inputs when gathering the activity data

All neurons measured in session 9 are removed due to issues when fetching the activity

In [16]:
session, scan_idx, unit_id, brain_area = (nda.AreaMembership & df).fetch('session','scan_idx','unit_id', 'brain_area')
df_areas = pd.DataFrame({'session': session, 'scan_idx': scan_idx, 'unit_id': unit_id, 'brain_area': brain_area})
df_areas = df_areas.loc[df_areas['session'] != 9]
df_areas

/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


,session,scan_idx,unit_id,brain_area
0,4,7,648,RL
1,4,7,662,V1
2,4,7,664,V1
3,4,7,665,RL
4,4,7,671,V1
...,...,...,...,...
8190,8,5,10489,V1
8191,8,5,10542,V1
8192,8,5,10550,RL
8193,8,5,10567,RL


## Area V1

In [18]:
df_area_v1 = df_areas.loc[df_areas['brain_area'] == 'V1']
df_area_v1

,session,scan_idx,unit_id,brain_area
1,4,7,662,V1
2,4,7,664,V1
4,4,7,671,V1
5,4,7,682,V1
6,4,7,703,V1
...,...,...,...,...
8185,8,5,10458,V1
8186,8,5,10463,V1
8189,8,5,10480,V1
8190,8,5,10489,V1


This loop follows the same procedure as the one described in the layers data gathering file. You can look there for the explanation.

In [22]:
list_area_v1 = []
for row in range():
    unit_key = {'session':df_area_v1.iloc[row]['session'], 'scan_idx': df_area_v1.iloc[row]['scan_idx'], 'unit_id': df_area_v1.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_v1.append(avg_activity)
data_area_v1 = np.array(list_area_v1)
np.savetxt('v1_subsection.csv', data_area_v1, delimiter= ',')
    

/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


## Area LM

In [17]:
df_area_lm = df_areas.loc[df_areas['brain_area'] == 'LM']
df_area_lm

,session,scan_idx,unit_id
0,4,7,664
1,4,7,912
2,4,7,985
3,4,7,1024
4,4,7,1055
...,...,...,...
4811,9,6,4929
4812,9,6,4964
4813,9,6,4984
4814,9,6,5071


In [55]:
list_area_lm = []
for row in range():
    unit_key = {'session': df_area_lm.iloc[row]['session'], 'scan_idx': df_area_lm.iloc[row]['scan_idx'], 'unit_id': df_area_lm.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_lm.append(avg_activity)
data_area_lm = np.array(list_area_lm)
np.savetxt('lm_subsection.csv', data_area_lm, delimiter= ',')

## Area AL

In [ ]:
df_area_al = df_areas.loc[df_areas['brain_area'] == 'AL']
df_area_al

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
8,12,t,115623132022390280,864691136521587601,4,7,2585,191.044803
9,13,t,115623201144775522,864691135593561643,4,7,2591,192.849116
10,15,t,110347193524762590,864691135382480602,4,7,2621,229.130676
11,17,t,111754568542850272,864691135467558284,4,7,2626,232.799003
12,18,t,110277031073449184,864691135683238898,4,7,2647,235.657765


In [ ]:
list_area_al = []
for row in range():
    unit_key = {'session': df_area_al.iloc[row]['session'], 'scan_idx': df_area_al.iloc[row]['scan_idx'], 'unit_id': df_area_al.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_al.append(avg_activity)
data_area_al = np.array(list_area_al)
np.savetxt('al_subsection.csv', data_area_al, delimiter= ',')

## Area RL

In [ ]:
df_area_rl = df_areas.loc[df_areas['brain_area'] == 'RL']
df_area_rl

,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,depth
8,12,t,115623132022390280,864691136521587601,4,7,2585,191.044803
9,13,t,115623201144775522,864691135593561643,4,7,2591,192.849116
10,15,t,110347193524762590,864691135382480602,4,7,2621,229.130676
11,17,t,111754568542850272,864691135467558284,4,7,2626,232.799003
12,18,t,110277031073449184,864691135683238898,4,7,2647,235.657765


In [ ]:
list_area_rl = []
for row in range():
    unit_key = {'session': df_area_rl.iloc[row]['session'], 'scan_idx': df_area_rl.iloc[row]['scan_idx'], 'unit_id': df_area_rl.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_rl.append(avg_activity)
data_area_rl = np.array(list_area_rl)
np.savetxt('rl_subsection.csv', data_area_rl, delimiter= ',')